In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame
from autoviz.AutoViz_Class import AutoViz_Class
from sklearn.model_selection import train_test_split

from utils.dataset import load_dataset

%matplotlib inline

In [ ]:
# Load dataset and describe it

dataset_path = 'dataset/hcvdat0.csv'


hcv_df = load_dataset(dataset_path)
hcv_df = hcv_df[hcv_df.columns[1:]]  # remove first 'Unnamed' column
hcv_df.head(5)

print(f'Number of different categories: {hcv_df["Category"].value_counts()}')
# describe DataFrame
hcv_df.describe()

# Identify non-numeric columns and their unique values
for column in ['Category', 'Sex']:
    unique_values = hcv_df[column].unique()
    print(f'Unique values in column {column}:')
    print(unique_values)
    print('\n')

In [ ]:
# Transforming string values to numeric (Category, Sex) and filling nulls
category_mapping = {
    '0=Blood Donor': 0,
    '1=Hepatitis': 1,
    '2=Fibrosis': 2,
    '3=Cirrhosis': 3,
    '0s=suspect Blood Donor': 4
}

sex_mapping = {
    'm': 0,
    'f': 1
}

replacement_dict = {
    'Category': category_mapping,
    'Sex': sex_mapping
}

hcv_df.replace(replacement_dict, inplace=True)
hcv_df = hcv_df.fillna(method='ffill')
hcv_df.head(5)

In [ ]:
# creating an AutoViz instance
AV = AutoViz_Class()

# generating data visualization automatically
AV.AutoViz(
    filename='',
    sep=',',
    depVar='',
    dfte=hcv_df,
    header=0,
    verbose=0,
    lowess=False,
    chart_format='svg',
    max_rows_analyzed=10000,
    max_cols_analyzed=30
)

In [ ]:
# Plot correlation between columns (features)

sns.heatmap(hcv_df.corr(), annot = True, annot_kws={'fontsize': 8})

In [ ]:
# Split train and test data
x_train, x_test, y_train, y_test = train_test_split(hcv_df, test_size=0.2, shuffle=True)

In [ ]:
# Create tabular predictor
y_label = 'Category'